In [1]:
import sys
import os

# # Add the directories containing detection_model.py and byte_tracker.py to the Python path
# sys.path.append(os.path.abspath(os.path.join('..', 'detection')))
# sys.path.append(os.path.abspath(os.path.join('..', 'tracker')))
#================================================================================================

sys.path.append("..") # back to root folder if you put the notebook here

In [ ]:
from configparser import ConfigParser
from detector.detector import Detector
from tracker.byte_tracker import BYTETracker
import cv2
import numpy as np

In [3]:
# SETUP
config = ConfigParser()
config.read('../tracker.cfg')

cap = cv2.VideoCapture(os.path.join("..", config.get('video', 'video_path')))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_tracking = cv2.VideoWriter(os.path.join("..", config.get('video', 'video_out_tracking')), cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(cv2.CAP_PROP_FPS)), (width, height))
out_detect = cv2.VideoWriter(os.path.join("..", config.get('video', 'video_out_detect')), cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(cv2.CAP_PROP_FPS)), (width, height))

aspect_ratio_thresh = 0.6 # more condition for vertical box if you like
min_box_area = 100 # minimum area of the trcking box to be considered

class_name = ['ball', 'goalkeeper', 'player', 'referee']

setup args for ByteTracker

In [4]:
class TrackerArgs:
    def __init__(self, track_thresh, track_buffer, match_thresh, fuse_score):
        self.track_thresh = track_thresh
        self.track_buffer = track_buffer
        self.match_thresh = match_thresh
        self.fuse_score = fuse_score
        self.mot20 = False

*for clean code purpose; for future practice pls consider others code structure

In [5]:
def draw_fps(frame):
    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = ' FPS: ' + str(fps) + ' Width: ' + str(cap.get(3)) + ' Height: ' + str(cap.get(4))
    cv2.putText(frame, fps, (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame

def draw_bbox(frame, id, x1, y1, x2, y2, conf, type='detect'):
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    if type == "detect":
        cv2.putText(frame, f"{class_name[id]}: {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    if type == "track":
        cv2.putText(frame, f'ID: {id}, Score: {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
def main():
    weights = "../weights/best.pt"
    model = Detector(weights)
    args = TrackerArgs(
        track_thresh=config.getfloat('Tracker', 'track_thresh'),
        track_buffer=config.getint('Tracker', 'track_buffer'),
        match_thresh=config.getfloat('Tracker', 'match_thresh'),
        fuse_score=config.getboolean('Tracker', 'fuse_score')
    )
    tracker = BYTETracker(args)
    frame_id = 0
    tracking_results = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break        
        detect_results = model.detect(frame)
        detections = []
        frame_detected = frame.copy()
        draw_fps(frame_detected)
        for result in detect_results:
            for boxes in result.boxes:
                # format tracker input
                label, conf, bbox = int(boxes.cls[0]), float(boxes.conf[0]), boxes.xyxy.tolist()[0]
                print(label, conf, bbox)
                x1, y1, x2, y2 = map(int, bbox)
                class_id = int(label)
                detections.append([x1, y1, x2, y2, conf]) #detections bbox for tracker
                
                draw_bbox(frame_detected, class_id, x1, y1, x2, y2, conf, type='detect')
        out_detect.write(frame_detected)

        
        # Convert detections to numpy array
        if detections:
            detections = np.array(detections)
        else:
            detections = np.empty((0, 5))
        
        # Update tracker with detections format, frame size, and test size
        online_targets = tracker.update(detections, [height, width], [height, width])
        # Draw tracked objects
        frame_tracked = frame.copy()
        for t in online_targets:
            tlwh = t.tlwh
            tid = t.track_id
            # vertical = tlwh[2] / tlwh[3] > aspect_ratio_thresh
            # if tlwh[2] * tlwh[3] > min_box_area and not vertical:
            if tlwh[2] * tlwh[3] > min_box_area:
                # save results
                tracking_results.append(
                    f"{frame_id},{tid},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},{t.score:.2f},-1,-1,-1\n"
                )
                # Draw the bounding box, score is the confidence, calc = IoU
                x1, y1, w, h = map(int, tlwh)
                x2, y2 = x1 + w, y1 + h
                draw_bbox(frame_tracked, tid, x1, y1, x2, y2, t.score, type='track')

        # Write and display the frame

        out_tracking.write(frame_tracked)
        cv2.imshow('frame', frame_tracked)
        # cv2.imshow('frame', frame_detected)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out_detect.release()
    out_tracking.release()
    cv2.destroyAllWindows()

In [7]:
main()


0: 384x640 1 goalkeeper, 18 players, 2 referees, 70.2ms
2 0.893754780292511 [328.82220458984375, 495.81585693359375, 363.8630676269531, 569.3467407226562]
2 0.8881829380989075 [223.20632934570312, 513.1036376953125, 257.1714172363281, 594.3099365234375]
2 0.8878268599510193 [1231.96044921875, 431.9402160644531, 1264.126220703125, 500.7411804199219]
2 0.8777821063995361 [1308.0133056640625, 448.4638366699219, 1349.730712890625, 517.6246337890625]
2 0.8716025352478027 [586.8529052734375, 589.427734375, 631.33642578125, 672.4623413085938]
2 0.8569434881210327 [531.9282836914062, 686.3278198242188, 582.200927734375, 784.8297729492188]
2 0.8565713763237 [375.1097412109375, 305.010986328125, 400.44720458984375, 365.756103515625]
2 0.856372058391571 [1572.337646484375, 611.4393310546875, 1616.0738525390625, 694.3416137695312]
2 0.8529829382896423 [875.52392578125, 361.0657653808594, 904.3905029296875, 424.7848205566406]
2 0.8510079979896545 [844.1797485351562, 635.6821899414062, 899.78924560